### Bandit Environment

the below code implements the k-armed bandit problem which navigates the exploration-exploitation dilemma:
- There are k slot machines standing in front of an agent.
- The agent has h turns, where they must pick a slot machine's arm to pull per turn.
- The slot machine either produces a 1 or 0 as the reward.
- Each slot machine has it's own reward probability distribution unknown to the agent.
- The goal is to maximize the total reward after h turns.

The optimal strategy to the problem challenges the agent to strike a balance of exploring the individual probability distributions of the different machines (exploration) and maximizing profits based on the information acquired so far (exploitation)

#### Properties:
- currently, only supporting a stationary environment. Future updates will occur to allow the option of a non-stationary environment.
    - (a stationary environment means that the underlying true probabilities do not change over time so in our case, a non-stationary environment would mean that with each turn. The arms underlying probabilities would shift over turns)
- As the case with the historical k-bandit problem, our output for a pull is either 'success' or 'failure', in the future there will be an option for a continous variable output so that more policies can be applicable to the bandit environment

In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class Bandit:


    def __init__(self, arms, turns, seed=None, dist='uniform', dist_params=None):
        '''
        Initialize a k-armed bandit environment. world state is automatically updated when pull(arm_index) is used.

        Parameters:
            arms (int): The number of arms (k) of the bandit
            turns (int): The total number of pulls (horizon)
            seed (int): random seed used for reproducibility (compare RL algorithms)
            dist (str): The distribution to generate true reward probabilities.
                    options: 'unform' (default) or 'beta'
            dist_params (dict, optional): Additional parameters for the distribution as needed.
                    For 'beta', you might use {'a' : 1, 'b' : 1}
        '''
        # set random seed
        if seed is not None:
            np.random.seed(seed)

        # Validate input parameters
        if arms <= 0 or turns <= 0:
            raise ValueError("The number of arms and turns must be positive integers.")

        self.k = arms                     # Number of arms (k)
        self.h = turns                    # Total number of pulls (horizon)
        self.turn = 1                     # Current turn
        self.scores = []                  # Store payout of different RL strategies

        # internal state:
        # Each element is a tuple (n, w): n = number of pulls, w = total reward for that arm
        # one tuple per arm
        self.state = [(0, 0) for _ in range(self.k)]

        # History recording: will store (turn_number, arm_index, reward)
        self.history = []

        # True reward probabilities (hidden from the learning agent)
        if dist == 'uniform':

            # Sample true reward probabilities uniformly between 0 and 1
            self.true_probs = np.random.rand(self.k)

        elif dist == 'beta':
            #Use a beta distribution, Set default Beta(1,1) if no parameters provided
            a = 1
            b = 1
            if dist_params is not None:
                a = dist_params.get('a', 1)
                b = dist_params.get('b', 1)
            self.true_probs = np.random.beta(a, b, size=self.k)
        else:
            raise ValueError("Distribution type not recognized. Use 'uniform' or 'beta'.")

    def reset(self):
        '''
        Reset the bandit's state and history to the initial condition.
        '''
        self.state = [(0, 0) for _ in range(self.k)]
        self.history = []
        self.turn = 1

    def pull(self, arm_index):
        '''
        Simulate pulling a specific arm.

        Parameters:
            arm_index (int): The index of the arm to pull.

        Returns:
            reward (int): The reward obtained (1 for success, 0 for failure).
        '''
        #validate # of turns
        if not (1 <= self.turn <= self.h):
            raise ValueError("turn must be between 1 and h.")

        #validate arm index
        if not (0 <= arm_index < self.k):
            raise ValueError("arm_index must be between 0 and k-1.")

        #determine if the pull results is a success by using a bernoulli distribution
        prob_success = self.true_probs[arm_index]
        reward = np.random.binomial(1, prob_success)

        #update the state for that arm
        n, w = self.state[arm_index]
        new_n = n + 1             # number of times arm has been pulled after this pull
        new_w = w + reward        # total reward for arm after this pull
        self.state[arm_index] = (new_n, new_w)

        #record the pull in history (turn, arm_index, reward)
        self.history.append({"turn": self.turn, "arm": arm_index, "payout" : reward})

        #print state after pull
        message = f"Turn {self.turn}: Pulled arm {arm_index}. "
        message += "Result: SUCCESS. " if reward else "Result: FAILURE. "
        message += f"Arm {arm_index} state: (#_of_pulls: {new_n}, total_reward: {new_w}). "

        print(message)
        self.turn += 1

        return reward

    def get_state(self):
        """
        Return the current state.
        """
        return list(self.state)

    def get_history(self):
        """
        Return the history of pulls.
        - (turn, arm_index, reward)
        """
        return list(self.history)

    def store_strat(self, name):
        """
        store payout of RL strategy on current bandit environment and calls get_scores()
        """
        payout = sum([score[1] for score in self.state])
        self.scores.append((name, payout))

        self.get_scores()                        #print performance of policy
        self.reset()                             #reset environment

    def get_scores(self):
        """
        prints the payouts of all RL strategies on current bandit environment
        """
        print('---------------------------------------------------------------------------')
        for name, payout in self.scores:
            print(f'strategy - {name} | total payout - {payout} | total turns - {self.h}')

### Example Usage of Bandit Class:
To showcase how to use the Bandit environment with a defined policy I created 'basic_greed()', a pure greedy policy that fully utilizes past experiences to dictate which arm to pull.
- The policy calculates the 'win-rate' (reward/pulls) of each arm and picks the highest
- When multiple arms have the highest winrate, a 'np.random.choice()' is made
- Due to the nature of the policy, the agent at first explores the different arms until it finds initial success, wherein the arm that yields success will consistently be chosen therafter

In [3]:
def basic_greed(world):
    '''
    basic pure greedy policy that aims to always pick the highest percentage chance of payout based on past experience.
    if there exist multiple options, randomly choose one using random.choice()

    purposefully leaving inherent randomness

    Parameters:
        world (Bandit): the environment in which we are utilizing strategy
    '''
    perc_chance = {}

    while world.turn <= world.h:                    # keep pulling till out of turns

        curr_state = world.get_state()              # Update current state

        for arm, (n, w) in enumerate(curr_state):   # calculate performance history of each arm - payout / #_of_pulls
            if n == 0:                              # Dont divide by 0
                perc_chance[arm] = 0
            else:
                perc_chance[arm] = w/n              # % chance of payout


        #filter arm(s) with highest chances
        high_chance_arms = [arm for arm, perc in perc_chance.items() if perc == max(perc_chance.values())]

        #randomly choose arm out of highest_chances
        world.pull(np.random.choice(high_chance_arms))

        #end of while loop

    print("\nCurrent state:")
    print(bandit_env.get_state())

    print("\nHistory:")
    print(bandit_env.get_history())

    print("\nPayout:")
    world.store_strat("Basic Greed")               #stores the performance of RL policy and resets environment

bandit_env = Bandit(arms=6, turns=15, seed=3, dist='beta', dist_params={'a': 1, 'b' : 1})

print("True reward probabilities (hidden):")
print(bandit_env.true_probs)

# Simulate pulling a few arms
basic_greed(bandit_env)


True reward probabilities (hidden):
[0.36284521 0.37732775 0.10454926 0.06138408 0.69978668 0.04114688]
Turn 1: Pulled arm 5. Result: FAILURE. Arm 5 state: (#_of_pulls: 1, total_reward: 0). 
Turn 2: Pulled arm 1. Result: FAILURE. Arm 1 state: (#_of_pulls: 1, total_reward: 0). 
Turn 3: Pulled arm 0. Result: FAILURE. Arm 0 state: (#_of_pulls: 1, total_reward: 0). 
Turn 4: Pulled arm 0. Result: FAILURE. Arm 0 state: (#_of_pulls: 2, total_reward: 0). 
Turn 5: Pulled arm 3. Result: FAILURE. Arm 3 state: (#_of_pulls: 1, total_reward: 0). 
Turn 6: Pulled arm 5. Result: FAILURE. Arm 5 state: (#_of_pulls: 2, total_reward: 0). 
Turn 7: Pulled arm 1. Result: FAILURE. Arm 1 state: (#_of_pulls: 2, total_reward: 0). 
Turn 8: Pulled arm 4. Result: FAILURE. Arm 4 state: (#_of_pulls: 1, total_reward: 0). 
Turn 9: Pulled arm 0. Result: SUCCESS. Arm 0 state: (#_of_pulls: 3, total_reward: 1). 
Turn 10: Pulled arm 0. Result: SUCCESS. Arm 0 state: (#_of_pulls: 4, total_reward: 2). 
Turn 11: Pulled arm 0. Re

### Upper-Confidence-Bound(UCB)
upper-bound-confidence interval for the bandit problem environment
A higher c value is used when exploration is more critical within an environment
- non-stationary environment
- large action space (many arms)
- uncertain environments where large variation in rewards

This algorithm is used for **Decision Making Under Uncertainty**
### UCB formula
$$ a_t = \mathop{\text{arg max}}_{\textbf{i}}\left(\hat{\mu_{i}}+c*\sqrt{\frac{\ln{t}}{n_i}}\right) $$
- $a_t$ : The action (arm) chosen at time $t$
- $\hat{\mu_{i}}$ : The mean reward
- $t$ : current turn
- $n_i$ : number of times arm $i$ has been pulled
- $c$ : exploration parameter


### Future Considerations
This policy does not perform well in a stationary Environment. While the logic is sound, it needs to be furthur tested once a non-stationary environment option is implemented to bandit

In [4]:
import math

def ucb(world, c=1.0, temp=False):
    """
    Parameters:
        world (Bandit): The environment in which we are utilizing strategy
        c (float): Value representing the Exploration Parameter, default 1.0 as a standard
        temp (bool): Determines if the user desires the exploration parameter to
    """
    while world.turn <= world.h:                           # Pull until horizon is reached
        curr_state = world.get_state()
        arm_conf = {}

        if temp:
            c = c / math.sqrt(world.turn)                      # Decay c over time

        for arm, (n, w) in enumerate(curr_state):
            mean_reward = w/n if n > 0 else 0                             # 'Winrate' (reward/#_of_pulls)
            t = world.turn
            # Exploration_val is set to infinity if a arm has not been chosen yet to ensure each arm is chosen at least once
            exploration_val = math.sqrt( math.log(t) / n ) if n > 0 else float('inf')                   # As we reach the horizon, arms that have not been pulled have a higher exploration_val
            uncertainty = c * exploration_val                                                           # Exploration parameter * exploration_val = uncertainty
            conf_val = mean_reward + uncertainty                                                        # (Exploitation + Exploration) quantified
            arm_conf[arm] = conf_val

        high_val = max(arm_conf.values())                                                               # Find highest confidence value out of all arms
        highest_arms = [arm for arm, conf_values in arm_conf.items() if conf_values == high_val]        # Filter arms that have highest confidence value
        chosen_arm = np.random.choice(highest_arms)                                                     # random arm chosen in case of tie-breaker
        world.pull(chosen_arm)

    world.store_strat("Upper Confidence Bound (UCB) Interval")


print("True reward probabilities (hidden):")
print(bandit_env.true_probs)

# Simulate pulling a few arms
ucb(bandit_env)

True reward probabilities (hidden):
[0.36284521 0.37732775 0.10454926 0.06138408 0.69978668 0.04114688]
Turn 1: Pulled arm 1. Result: FAILURE. Arm 1 state: (#_of_pulls: 1, total_reward: 0). 
Turn 2: Pulled arm 3. Result: FAILURE. Arm 3 state: (#_of_pulls: 1, total_reward: 0). 
Turn 3: Pulled arm 0. Result: FAILURE. Arm 0 state: (#_of_pulls: 1, total_reward: 0). 
Turn 4: Pulled arm 2. Result: FAILURE. Arm 2 state: (#_of_pulls: 1, total_reward: 0). 
Turn 5: Pulled arm 4. Result: SUCCESS. Arm 4 state: (#_of_pulls: 1, total_reward: 1). 
Turn 6: Pulled arm 5. Result: FAILURE. Arm 5 state: (#_of_pulls: 1, total_reward: 0). 
Turn 7: Pulled arm 4. Result: SUCCESS. Arm 4 state: (#_of_pulls: 2, total_reward: 2). 
Turn 8: Pulled arm 4. Result: SUCCESS. Arm 4 state: (#_of_pulls: 3, total_reward: 3). 
Turn 9: Pulled arm 4. Result: SUCCESS. Arm 4 state: (#_of_pulls: 4, total_reward: 4). 
Turn 10: Pulled arm 4. Result: SUCCESS. Arm 4 state: (#_of_pulls: 5, total_reward: 5). 
Turn 11: Pulled arm 4. Re

## 2.1.1 Bayesian Dynamic Programming

### Explanation of Algorithm

The following algorithm is a Reinforcement Learning policy that utilizes Bayesian reasoning and a variant of the bellman equation to determine the optimal arm to pull that results in the highest total payoff.

We start by using a uniform beta distribution (prior) to represent each arm's underlying true probability distribution
- Using observations from the environment, we update the parameters of our beta distributions to form a probability distribution that is more representative of the true underlying probability distribution (posterior) of each arm.

To decide which arm to pull in the real environment, we calculate the total expected Value for each arm: $$\text{Expected Value}_i = p_i \times (R + V_{\text{succ}}) + (1 - p_i) \times (0 + V_{\text{fail}})$$
- $R$ : immediate reward upon success
- $p_i$ : the probability of success (beta mean)
- $V_{succ}$ : the future value if successful
- $V_{fail}$ : the future value if not successful

The expected value is calculated by taking a weighted sum of the two possible outcomes (success and failure), the weights are determined by taking the beta mean of the respective arm's estimated probability distribution.

$$\text{beta mean}=\frac{w+1}{n+2}$$  
- $w$ : number of pulls that resulted in reward
- $n$ : total number of pulls

Utilizing the recursive nature of the dynamic programming approach we calculate the expected value from the current turn to when $\text{turn} = 0$, by simulating all possible actions and their respective outcomes (success, failure). Once all the recursive calls reach their base case, we choose the arm with the highest total expected value from the current turn.  

We then interact with our environment to pull said arm, using the results we update our arm's beta distribution and calculate the total expected values again for each arm using our updated beta distribution. The agent continues until out of turns.

### Understanding the beta distribution

Parameters :
- a (alpha) : Interpreted as the "prior count" of successes      (evidence for the event occuring)
- b (beta)  : similarly, viewed as the "prior count" of failures (evidence against the event)

- When we set both parameters to 1, (i.e. Beta(1,1)) we yield a uniform distribution over interval (0,1) that does not favor any probability value over another

- Increasing alpha relative to beta shift the distribution towards the higher probability values, reflecting prior belief or evidence that successes are more likely

- As more observations are collected, the parameters are updated (alpha increases with successes, beta increases with failures) and the distribution comes closer to the true underlying probability

Methods of Updating after Observation:
- Historically, alpha would be increased by 1 when a success is observed and beta the same when a failure is observed.
- In the case of a non-stationary environment, we may implement a learning rate when updating the beta distribution parameters.
    - If the learning rate is changed over time we can model varying true probability distributions (i.e. if we slowly increase the learning rate over turns, we can "fade out" past observations. Modeling an environment where future observations are more significant then initial ones)

### Future Considerations:

The current memoization strategy persists across turns. Which in a stationary environment is valid because our memoization key (tuple(state), remaining_pulls) fully encapsulates the relevant Belief (via counts of success and failures). If we were dealing with a non-stationary environment where there exists external factors that change the dynamics, then the cached value might be out-of-date. Currently, the state (n, w) fully summarizes the underlying beliefs for the standard finite-horizon k-bandit problem.

The algorithm is computationally expensive, with a big O notation of $O((2k)^h)$ where $k$ represents the number of arms and $h$ is our horizon. The base of the exponent is $2k$ because for reach arm we are calling two recursive calls (success, failure). In the future, pruning methods should be employed to lower the computation cost. As of now, the highest horizon we can test on the algorithm is $h=15$, any more turns result in a runtime that can last up to 40 minutes - 4 hours.

In [5]:
def bay_rec(world):
    """
    a bayesian policy that uses dynamic programming to find the optimal arm to pull based on expected future reward.

    Parameters:
        world (bandit): the environment in which we are utilizing strategy
    """
    def updateState(state, arm, result):
        """
        function that returns an updated state given an arm_index and the result (success = 'True', failure = 'False')

        Parameters:
            state  (list): list of tuples (n, w) representing - n (total pulls), w (successful pulls)
            arm     (int): an integer representing the arm_index to be updated
            result (bool): boolean representing success or failure of pull

        Returns:
            updated state (list): the updated state after pull
        """
        n, w = state[arm]
        state[arm] = (n + 1, w + result)
        return state
        #end of updateState

    def value(val_map, state, remaining_pulls):
        """
        Value function that recurses until base condition is reached (V(state) = 0).

        Parameters:
            val_map           (dic): dictionary of world states : expected future reward
            state            (list): list of tuples (n, w) representing - n (total pulls), w (successful pulls)
            remaining_pulls   (int): the number of remaining pulls

        Returns:
            value             (int): total expected future Value given pulling arm_i
            arm_index         (int): represents arm associated with value
        """
        if remaining_pulls == 0:                             # Base case: if no pulls remain, return 0 and invalid arm index
            return (0, -1)

        state_key = tuple(state)                             # Store (state, remaining_pulls) in val_map
        if (state_key, remaining_pulls) in val_map:
            return val_map[(state_key, remaining_pulls)]

        best_val = -float('inf')
        best_arm = None                                     # Initialize best_arm with a default value
        for arm, (n, w) in enumerate(state):

            #Bayesian update:
            p_i = (w + 1) / (n + 2)                         # Expected success probability (beta mean) for arm_i

            # Use copies of the original state so as to preserve original state
            # Simulate success:
            state_success = updateState(state.copy(), arm, True)  # Simulate arm pull result in success
                                                            # Compute future Value based on success outcome
            value_success = value(val_map, state_success, remaining_pulls - 1)[0]
            # Simulate failure:
            state_failure = updateState(state.copy(), arm, False)  # Simulate arm pull result in failure
                                                            # Compute future Value based on failure outcome
            value_failure = value(val_map, state_failure, remaining_pulls - 1)[0]
                                                            # Total Expected Value
                                                            # For success: reward = 1 (plus future reward)
                                                            # For failure: reward = 0 (plus future reward)
            curr_val = p_i * (1 + value_success) + (1 - p_i) * (0 + value_failure)

            if curr_val > best_val:                         # Determine the arm with the highest Total Expected Value
                best_val = curr_val
                best_arm = arm

                                                            # Cache the computed value for this state and remaining pulls
        val_map[(state_key, remaining_pulls)] = (best_val, best_arm)
        return best_val, best_arm
        #end of value

    val_map = {}                                           # Val_map, persists across turns

    while world.turn <= world.h:                           # For every turn until horizon is reached

        curr_state = world.get_state()                     # Get current state from environment (Updated)
                                                           # Compute best value and arm for the current state and remaining pulls
        best_val, best_arm_index = value(val_map, curr_state, world.h - world.turn + 1)
                                                           # Validate that best_val_arms contain at least one decision
        if best_arm_index is None or best_arm_index == -1:
            raise ValueError("No best arm was found. Check value function logic.")

        world.pull(best_arm_index)                         # interact with real environment

    world.store_strat("Bayesian_Bellman")                  # Store performance and reset Environment


print("True reward probabilities (hidden):")
print(bandit_env.true_probs)

bay_rec(bandit_env)

True reward probabilities (hidden):
[0.36284521 0.37732775 0.10454926 0.06138408 0.69978668 0.04114688]
Turn 1: Pulled arm 0. Result: SUCCESS. Arm 0 state: (#_of_pulls: 1, total_reward: 1). 
Turn 2: Pulled arm 0. Result: SUCCESS. Arm 0 state: (#_of_pulls: 2, total_reward: 2). 
Turn 3: Pulled arm 0. Result: FAILURE. Arm 0 state: (#_of_pulls: 3, total_reward: 2). 
Turn 4: Pulled arm 1. Result: SUCCESS. Arm 1 state: (#_of_pulls: 1, total_reward: 1). 
Turn 5: Pulled arm 1. Result: FAILURE. Arm 1 state: (#_of_pulls: 2, total_reward: 1). 
Turn 6: Pulled arm 2. Result: FAILURE. Arm 2 state: (#_of_pulls: 1, total_reward: 0). 
Turn 7: Pulled arm 3. Result: FAILURE. Arm 3 state: (#_of_pulls: 1, total_reward: 0). 
Turn 8: Pulled arm 4. Result: SUCCESS. Arm 4 state: (#_of_pulls: 1, total_reward: 1). 
Turn 9: Pulled arm 4. Result: SUCCESS. Arm 4 state: (#_of_pulls: 2, total_reward: 2). 
Turn 10: Pulled arm 4. Result: SUCCESS. Arm 4 state: (#_of_pulls: 3, total_reward: 3). 
Turn 11: Pulled arm 4. Re